<a href="https://colab.research.google.com/github/prasys/lightstay-automation/blob/main/Hilton_LightStay_Automation_Project_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-Installation - Installing Chrome

Here we will be installing Selenium and Chrome together. Selenium is a web automation tool to simulate click and a lot of other things. We will also be instlaling Chrome

#Import phases

Here we will be importing libarires and a lot of other configuration, please make sure that you change the filename `filename.csv` to something else.

We are fooling the system to have a look and thinking that it is a chrome browser running on a full screen on Linux. It's to trick the system that it is a genuine human with a real computer clicking it through!

In [1]:
from selenium import webdriver


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  # Import TimeoutException
import time
import pandas as pd
from tqdm import tqdm
import datetime

## FILE CONFIGURATION - CURRENTLY THE FILENAME IS SET TO FILEOUT.CSV
df = pd.read_csv('Aug24.csv', encoding='latin1')
#df = pd.read_csv('may24_subset.csv', encoding='utf-8')
#FOR OUR LOOKUP TABLE
df_lookup = pd.read_csv('lookup.csv')
#df_lookup['State'] = df_lookup['State'].replace('Fiji', 'Queensland')


# DATA CLEANING!!
df.columns = df.columns.str.strip()
progress_bar = tqdm(total=len(df), position=0, leave=True)
df = df.replace('-','0')
df.replace('-', 0, inplace=True)
df = df.replace(' -   ','0')
df = df.fillna(0)
df_lookup = df_lookup.fillna(0)
df['Meeting /Event Name'] = df['Meeting /Event Name'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # fix the names
df['Meeting /Event Name'] = df['Meeting /Event Name'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
df['Meeting /Event Name'] = df['Meeting /Event Name'].str.title().str.replace(' ', '') # convert to CamelCase


  0%|          | 0/65 [00:00<?, ?it/s]

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-software-rasterizer')  # Add this line
options.add_argument('--remote-debugging-port=9222')  # Add this line
options.add_argument("--verbose")

#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


driver = webdriver.Chrome(options=options)


# Navigate to the desired webpage
driver.get('https://lightstay.ei3.com/')

In [3]:
# Wait for the page to load (optional but recommended)
driver.implicitly_wait(10)  # wait up to 10 seconds

USER = "enteruser"
PASSWORD = "password"

# Locate the username and password fields
username_field = driver.find_element(By.ID, 'loginform:username')
password_field = driver.find_element(By.ID, 'loginform:password')
driver.execute_script("arguments[0].value='your_password';", password_field)



username_field.send_keys('cehmann')
driver.execute_script("arguments[0].value='adelaide';", password_field)
# Locate and click the login button
login_button = driver.find_element(By.ID, 'loginform:loginButton')
login_button.click()


#Phase 1 - Data Entry and Mark it as submitted

Here, we will be doing the following


*   Read the file (which contains hotel details) and the code
*   For each row, input the details on the fields and mark it as 'Submitted'
*   Save it

In [ ]:
import xlrd

for index, row in df.iterrows():
    driver.get('https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator/new')
    driver.find_element(By.ID, 'meetCalcForm:meetingName').send_keys(row['Meeting /Event Name'])
    driver.find_element(By.ID, 'meetCalcForm:recipientCompany').send_keys(row['Client / Account Name'])
    driver.find_element(By.ID, 'meetCalcForm:recipientName').send_keys(row['Client / Account Contact'])
    driver.find_element(By.ID, 'meetCalcForm:hotelName').send_keys(row['Hotel Name'])
    driver.find_element(By.ID, 'meetCalcForm:rfpNumber').send_keys(row['RFP number']) # fix RFP Number
#    driver.find_element(By.ID, 'meetCalcForm:meetingStatus').send_keys(row['Meeting Status'])
    date_value = row['Meeting/Event Start date']
    if isinstance(date_value, str):
        date_object = datetime.datetime.strptime(date_value, "%d/%m/%Y")
    elif isinstance(date_value, (int, float)):  # Excel date numbers are float
        date_object = xlrd.xldate.xldate_as_datetime(date_value, datemode=0)  # 0: 1900-based, 1: 1904-based
    else:
        print(f"Unexpected date value: {date_value}")
        continue  # Skip this row and move to the next one
    formatted_date_value = date_object.strftime("%m/%d/%Y")
    js_script = "document.getElementById('meetCalcForm:eventDateInputDate').value = '{}';".format(formatted_date_value)
    driver.execute_script(js_script)
    driver.find_element(By.ID, 'meetCalcForm:numberGuestNights').send_keys(row['Room Block w meeting'])
    
    # new fields - free text
    #driver.find_element(By.ID, 'meetCalcForm:agency').send_keys(row['Agency'])
    driver.find_element(By.ID, 'meetCalcForm:marketSegment').send_keys(row['Market Segment'])

    driver.find_element(By.ID, 'meetCalcForm:onPeakNights').send_keys(row['Number of room nights booked on peak night w meeting'])
    driver.find_element(By.ID, 'meetCalcForm:meetingFloorSpace').send_keys(row['Meeting/Event Floor Space'])
    driver.find_element(By.ID, 'meetCalcForm:meetingDuration').send_keys(row['Meeting room Duration'])
    attendees_value = row['Number of atttendees -  Meeting']
    js_script = f"document.getElementById('meetCalcForm:numberGuests').value = '{attendees_value}';"
    driver.execute_script(js_script)
    brekkie_value = row['Breakfast']
    js_script = f"document.getElementById('meetCalcForm:breakfasts').value = '{brekkie_value}';"
    driver.execute_script(js_script)
    lunch_value = row['Lunch']
    js_script = f"document.getElementById('meetCalcForm:lunches').value = '{lunch_value}';"
    driver.execute_script(js_script)
    dinner_value = row['Dinner']
    js_script = f"document.getElementById('meetCalcForm:dinners').value = '{dinner_value}';"
    driver.execute_script(js_script)
    snacks_value = row['Break']
    js_script = f"document.getElementById('meetCalcForm:snacks').value = '{snacks_value}';"
    driver.execute_script(js_script)
    # driver.find_element(By.ID, 'meetCalcForm:hotelContactName').send_keys(row['Hotel Contact Name'])
    # driver.find_element(By.ID, 'meetCalcForm:hotelNameCoverPage').send_keys(row['Hotel Name (Cover Page)'])
    # driver.find_element(By.ID, 'meetCalcForm:hotelContactEmail').send_keys(row['Hotel Contact Email'])
    # driver.find_element(By.ID, 'meetCalcForm:hotelContactPhone').send_keys(row['Hotel Contact Telephone Number'])
    # driver.find_element(By.ID, 'meetCalcForm:titlePageCheckbox').send_keys(row['Title Page'])
    # driver.find_element(By.ID, 'meetCalcForm:reportUnits').send_keys(row['Report Units'])
    #force few things
    driver.execute_script("document.getElementById('meetCalcForm:spaceUnitsselValue').value = 'm2';")
    driver.execute_script("document.getElementById('meetCalcForm:spaceUnitsInput').value = 'm²';")
    driver.execute_script("""
        document.getElementById('meetCalcForm:meetingStatusselValue').value = '1';
        document.getElementById('meetCalcForm:meetingStatusInput').value = 'Submitted';
    """)
    driver.execute_script("document.getElementById('meetCalcForm:durationTypeselValue').value = 'hours';")
    driver.execute_script("document.getElementById('meetCalcForm:durationTypeInput').value = 'Hours';")
    # Submit the form after filling it out (assuming there's a submit button)
    element = driver.find_element(By.ID, 'meetCalcForm:saveReportButton')
    driver.execute_script("arguments[0].click();", element)
    current_url = driver.current_url
    expected_url = 'https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator/submitconfirm'
    if current_url == expected_url:
      print("Entered Correctly!")
      progress_bar.update()
    else:
      print(f"Unexpected URL: {current_url}")
      print(row['Meeting /Event Name'])
      with open('stage1failed.txt', 'a') as f:
        f.write(row['Meeting /Event Name'] + '\n')


    # Optionally, add a delay to avoid overwhelming the server
    time.sleep(1)



#Phase 2 - Mark Status as Completed

Here, we will be doing the following


*   Read the file (which contains hotel details)
*   Do a search (based on the criteria 'Submitted') and the name
*   Select the first element and click on the link
*   Mark it as Completed




In [ ]:
# Assuming 'driver' is already defined and imported along with other necessary modules

progress_bar = tqdm(total=len(df), position=0, leave=True)
failed_meetings = []

for index, row in df.iterrows():
    try:
        print(str(row['Hotel Code']))
        driver.get('https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator?statusFilter=1')

        # Ensure 'Submitted' is checked
        submitted_checkbox = driver.find_element(By.ID, 'meetingForm:statusFilter:1')
        if not submitted_checkbox.is_selected():
            submitted_checkbox.click()

        # Enter the meeting/event name in the search box
        search_box = driver.find_element(By.ID, 'meetingForm:txtFilterText')
        search_box.clear()
        search_box.send_keys(row['Meeting /Event Name'])
        print(row['Meeting /Event Name'])
        search_box.send_keys(Keys.RETURN)
        driver.implicitly_wait(2)

        first_row = driver.find_element(By.ID, 'meetingForm:meetingTable:0')

        # Processing the first row
        meeting_name_link = first_row.find_element(By.XPATH, './/a[contains(@onclick, "jsf.util.chain")]')
        driver.execute_script("arguments[0].click();", meeting_name_link)

        # Inject code
        driver.execute_script("""
            document.getElementById('meetCalcForm:meetingStatusselValue').value = '2';
            document.getElementById('meetCalcForm:meetingStatusInput').value = 'Completed';
        """)

        element = driver.find_element(By.ID, 'meetCalcForm:saveReportButton')
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
    except Exception as e:
        print(f"Failed to process meeting: {row['Meeting /Event Name']} due to error: {e}")
        failed_meetings.append(row['Meeting /Event Name'])

    progress_bar.update()

progress_bar.close()

# Print the list of failed meetings
print("Failed Meetings:")
for meeting in failed_meetings:
      with open('stage2failed.txt', 'a') as f:
        f.write(meeting + '\n')

# Consider closing the driver at the end of the process
# driver.close()

# Phase 3

In [10]:
failed_meetings = []
df_lookup = pd.read_csv('lookup.csv')


progress_bar = tqdm(total=len(df), position=0, leave=True)
#special cleaning
#regex = r'.*[-@!\'](.*)'
#extracted = df['Meeting /Event Name'].str.extract(regex, expand=False)
#df['Meeting /Event Name'] = extracted.fillna(df['Meeting /Event Name'])
df['Meeting /Event Name'] = df['Meeting /Event Name'].str.lstrip()  # remove leading spaces

#merge our df to include additional details, so we can easily fill in the rows/columns


# Merge the dataframes based on 'id'
result = df.merge(df_lookup, on='Hotel Code', how='left')


for index, row in result.iterrows():
  print(str(row['Hotel Code']) + " " + str(row['Meeting /Event Name']))
  driver.get('https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator?statusFilter=2')
  #page_source = driver.page_source

# Print the page source
 # print(page_source)



  # Ensure 'Submitted' is checked
  try:
    submitted_checkbox = driver.find_element(By.ID, 'meetingForm:statusFilter:2')
    if not submitted_checkbox.is_selected():
        submitted_checkbox.click()  # This will check the 'Submitted' checkbox if it isn't already checked
  except Exception as e:
     # print(e)
      print(f"Step 1 Failed: {row['Meeting /Event Name']} due to error! SKIPPING")
      failed_meetings.append(row['Meeting /Event Name'])
      print("Will continue on to process the next one")
      #continue

  # Enter the meeting/event name in the search box
  search_box = driver.find_element(By.ID, 'meetingForm:txtFilterText')
  search_box.clear()  # Clear any existing text

  search_box.send_keys(row['Meeting /Event Name'])

  # Trigger the search by pressing Enter
  search_box.send_keys(Keys.RETURN) # Stimulate Enter and wait
  driver.implicitly_wait(10) # wait for 2 seconds

  try:
    element = driver.find_element(By.XPATH, "//input[@id='meetingForm:meetingTable:0:meetingActionSelectInput']")
    driver.execute_script("arguments[0].click();", element)

    element = driver.find_element(By.XPATH, "//div[@id='meetingForm:meetingTable:0:meetingActionSelectItem1']")
    driver.execute_script("arguments[0].click();", element)

    wait = WebDriverWait(driver, timeout=10)
    iframe_element = wait.until(EC.presence_of_element_located((By.ID, 'climeCoForm')))
    driver.switch_to.frame(iframe_element)

  # element = driver.find_element(By.XPATH, "//input[@id='meetingForm:meetingTable:0:meetingActionSelectInput']")
  # driver.execute_script("arguments[0].click();", element)
 #   try:
 #     element = driver.find_element(By.XPATH,"//div[@class='cky-consent-bar']")
 #     accept_button = driver.find_element_by_xpath("//button[contains(@class, 'cky-btn-accept')]")
 #     driver.execute_script("arguments[0].click();", accept_button)
    #  accept_button.click()


#    except Exception as e:
#      print("Not found - continue")


   # checkout_button = driver.find_element(By.XPATH, "//a[contains(text(),'Checkout Now')]")
  #  checkout_button.click()
  except Exception as e:
        print(f"Step 2 Failed : {row['Meeting /Event Name']} due to error! SKIPPING")
        failed_meetings.append(row['Meeting /Event Name'])
        continue

  # try:
  #     wait = WebDriverWait(driver, 10)
  #     accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All')]")))
  #     accept_button.click()
  #     print("Cookie consent accepted.")
  # except Exception as e:
  #     print(f"Error accepting cookies: {e}")
  #     continue
      
  
  try:
      wait = WebDriverWait(driver, 5)
      checkout_button = driver.find_element(By.XPATH, "//a[contains(text(),'Checkout Now')]")
      checkout_button.click()
  except Exception as e:
      print(f"Error accepting cookies: {e}")


  # this is where, we will populate the things for our form


  driver.implicitly_wait(10)

  # try:
  #     wait = WebDriverWait(driver, 10)
  #     accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All')]")))
  #     accept_button.click()
  #     print("Cookie consent accepted.")
  # except Exception as e:
  #     print(f"Error Found - Handling gracefully {e}")
  #     continue


  driver.find_element(By.ID, 'billing_first_name').clear()
  driver.find_element(By.ID, 'billing_first_name').send_keys(row['Hotel Code'])
 # district = driver.find_element(By.ID, "billing_first_name")
 # print(district.get_attribute("value"))
  driver.find_element(By.ID, 'billing_last_name').clear()
  driver.find_element(By.ID, 'billing_last_name').send_keys(row['Hotel Code'])
  driver.find_element(By.ID, 'billing_company').clear()
  driver.find_element(By.ID, 'billing_company').send_keys(row['Company'])



  country_name = row['Coiuntry/Region']
  select2_element = driver.find_element(By.CSS_SELECTOR, ".select2-selection")
  select2_element.click()

  # Wait for the desired option to be visible, then click it
  desired_option_xpath = f"//li[text()='{country_name}']"
  desired_option = wait.until(EC.visibility_of_element_located((By.XPATH, desired_option_xpath)))
  desired_option.click()
  wait = WebDriverWait(driver, timeout=10)
  iframe_element = wait.until(EC.presence_of_element_located((By.ID, 'billing_city')))

  driver.find_element(By.ID, 'billing_address_1').clear()
  driver.find_element(By.ID, 'billing_address_1').send_keys(row['Street address'])
  # add a check here to determine if it's Australia or New Zealand



  driver.find_element(By.ID, 'billing_city').clear()
  driver.find_element(By.ID, 'billing_city').send_keys(row['Suburb / City'])
  try:
    select2_container = driver.find_element(By.ID, "select2-billing_state-container")
    select2_container.click()
  except Exception as e:
      print("Error -Skipping")

  state_name = row['State']

  desired_option_xpath = f"//li[text()='{state_name}']"
  driver.implicitly_wait(20)
  try:
   desired_option = wait.until(EC.visibility_of_element_located((By.XPATH, desired_option_xpath)))
   desired_option.click()
  except TimeoutException as e:
    print(f"Timeout while waiting for element: {desired_option_xpath}")
    print("Injecting the option - Directly for these ones that are outside")
    driver.find_element(By.ID, 'billing_state').send_keys(row['State'])
    
  wait = WebDriverWait(driver, timeout=50)
  iframe_element = wait.until(EC.presence_of_element_located((By.ID, 'billing_postcode')))


  driver.find_element(By.ID, 'billing_postcode').clear()
  driver.find_element(By.ID, 'billing_postcode').send_keys(str(int(row['Postcode'])))
  driver.find_element(By.ID, 'billing_phone').clear()
  driver.find_element(By.ID, 'billing_phone').send_keys(str(int(row['Phone'])))
  driver.find_element(By.ID, 'billing_email').clear()
  driver.find_element(By.ID, 'billing_email').send_keys(str(row['email'].strip()))
  driver.find_element(By.ID, 'billing_city').clear()
  driver.find_element(By.ID, 'billing_city').send_keys(row['Suburb / City'])

 # element = driver.find_element(By.XPATH,"//div[@class='cky-consent-bar']")
 # accept_button = driver.find_element(By.XPATH, "//button[contains(@class, 'cky-btn-accept')]")
 # driver.execute_script("arguments[0].click();", accept_button)
  #driver.find_element(By.ID, 'billing_email').send_keys(str('test@test.com'))

  try:
    #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "affiliate_consent"))).click()
    checkbox = driver.find_element(By.ID, "affiliate_consent")
    checkbox.click()
    # driver.execute_script("arguments[0].click();", checkbox)
  except Exception as e:
      print("Error")
      print("Retrying")
      wait = WebDriverWait(driver, 10)
      checkbox = wait.until(EC.element_to_be_clickable((By.ID, "affiliate_consent")))
      driver.execute_script("arguments[0].click();", checkbox)

     # checkbox = driver.find_element(By.ID, "affiliate_consent")
     # checkbox.click()
      #driver.execute_script("arguments[0].click();", checkbox)

  #checkbox = driver.find_element(By.ID, "affiliate_consent")
  #checkbox.click()

 # except Exception as e:
 #   failed_meetings.append(row['Meeting /Event Name'])
 #   continue


  place_order_button = driver.find_element(By.ID, "place_order")
  driver.execute_script("arguments[0].click();", place_order_button)
  driver.implicitly_wait(5) # wait for 5 seconds

  # try:
  #   # Find the element by class name
  #   error_element = driver.find_element(By.CLASS_NAME, 'woocommerce-error')

  #   # Print the text content of the element
  #   print(error_element.text)
  #   print(f"Step 3 Failed: {row['Meeting /Event Name']} due to error: {e}")
  #   failed_meetings.append(row['Meeting /Event Name'])
  #   continue
  # except Exception as e:
  #   print("No error found continue")
  #   continue


  try:
      # Wait for error messages to be present in the DOM
      wait = WebDriverWait(driver, 10)
      error_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'woocommerce-error')))

      # Collect all error messages
      errors = [error.text for error in error_elements]
      error_message = "\n".join(errors)

      # Log the error message
      if error_message:
          print("Errors encountered:")
          print(error_message)
          print(f"Step 3 Failed: {row['Meeting /Event Name']} due to error: {error_message}")
          failed_meetings.append(row['Meeting /Event Name'])
      else:
          print("No errors found, but exception was raised.")
  except Exception as e:
      print(f"No error found continue: {str(e)}")
      continue





  try:
    element = WebDriverWait(driver, 10).until(
        EC.text_to_be_present_in_element((By.ID, "mb-10"), "Thank you! Your funds have been received!")
    )
    print("Text is present!")
  except TimeoutException:
    print("Text is not present or page did not load within 10 seconds.")

  driver.switch_to.default_content() # switch back

  driver.execute_script('document.getElementById("meetPurchaseOffsetsForm:carbonOffsetsPurchased").click();')


progress_bar.close()
#driver.close()


  0%|          | 0/65 [00:16<?, ?it/s]


ADLHI LmgCommercialAssetFinancePdDay
Error
Retrying
No error found continue: Message: 

ADLHI FamilyLawIntensiveConference2024
Error
Retrying
No error found continue: Message: 

ADLHI AptTravelEventJuly2024
Error
Retrying
No error found continue: Message: 

ADLHI KarstensMeeting
Error
Retrying
No error found continue: Message: 

ADLHI MacManagementWorkshop
Error
Retrying
No error found continue: Message: 

ADLHI NationalApprenticeEmploymentNetworkConference2024
Error
Retrying
No error found continue: Message: 

AKLHI NnzFineWineTourAuckland
Error
Retrying
No error found continue: Message: 

AKLHI 4SquareAnnualConference
Step 2 Failed : 4SquareAnnualConference due to error! SKIPPING
AKLHI IodGoverningAiForum
Error
Retrying
No error found continue: Message: 

AKLHI NzacsPjAwards2024
Error
Retrying
No error found continue: Message: 

AKLKA AminzConference
Error
Retrying
No error found continue: Message: 

AKLKA IcehouseLdp59Block1
Error
Retrying
No error found continue: Message: 

ALHGI C

  0%|          | 0/65 [29:50<?, ?it/s]

No error found continue: Message: 

